<h3>This file merges the GAN and sensor data. Not only it is needed for algorithm but also for comparison between the two files </h3>

In [1]:
#packages

import pandas as pd
import numpy as np
import boto3

In [11]:
#file import from S3

s3 = boto3.client('s3')

bucket = 'flood-prediction-master-dataset'
key = 'ml-sensor-data-1-hr/ml_sensor_data_1_hr.csv'

obj = s3.get_object(Bucket= bucket,Key= key)
sensor_file = pd.read_csv(obj['Body'],names=["time","river","rain","temperature","wind_direction","wind_speed"])

key = 'ml-sensor-data-1-hr/gan_data_1_hr.csv'

obj = s3.get_object(Bucket= bucket,Key= key)
gan_file = pd.read_csv(obj['Body'])  #GAN was saved with header. hence no need for stating column names

In [12]:
gan_file

,time,river,rain,temperature,wind_direction,wind_speed
0,2020-07-11T14,7.441,0.0,18.21,268.0,6.231
1,2020-07-11T13,6.743,0.2,17.44,268.0,5.341
2,2020-07-11T12,6.189,0.0,17.50,266.8,5.007
3,2020-07-11T11,5.234,0.0,17.22,263.3,5.929
4,2020-07-11T10,4.028,0.0,16.96,264.3,5.898
...,...,...,...,...,...,...
687,2020-06-12T04,7.920,0.0,10.77,10.5,4.524
688,2020-06-12T03,8.036,0.0,11.64,216.8,2.554
689,2020-06-12T02,7.881,0.0,13.88,8.3,3.607
690,2020-06-12T01,7.077,0.0,9.99,7.5,4.806


In [13]:
sensor_file

,time,river,rain,temperature,wind_direction,wind_speed
0,2020-07-11T14,7.659,0.0,17.15,268.0,6.231
1,2020-07-11T13,7.019,0.0,16.45,268.0,5.341
2,2020-07-11T12,5.659,0.0,16.62,266.8,5.007
3,2020-07-11T11,3.978,0.0,16.78,263.3,5.929
4,2020-07-11T10,2.690,0.0,16.24,264.3,5.898
...,...,...,...,...,...,...
687,2020-06-12T04,7.920,0.0,12.30,10.5,4.524
688,2020-06-12T03,8.036,0.0,12.39,216.8,2.554
689,2020-06-12T02,7.881,0.0,12.35,8.3,3.607
690,2020-06-12T01,7.077,0.0,12.08,7.5,4.806


In [14]:
#converting the string datatype of time values to datetime

sensor_file['timerecorded'] = pd.to_datetime(sensor_file['time'])
gan_file['timerecorded'] = pd.to_datetime(gan_file['time'])

In [15]:
#dropping the redundant column

sensor_file.drop(['time'],axis=1,inplace=True)
gan_file.drop(['time'],axis=1,inplace=True)

In [16]:
#sorting by datetime column

gan_file.sort_values(by=['timerecorded'],inplace=True)
sensor_file.sort_values(by=['timerecorded'],inplace=True)

In [17]:
#quicksight imports all data from all the csv files from the folder and files specified.
#Hence, a distinguishing column is required.

gan_file['source'] = 'GAN'
sensor_file['source'] = 'SENSOR'

In [18]:
#merge both datasets

final_file_1_hr = gan_file.append(sensor_file)

In [19]:
final_file_1_hr

,river,rain,temperature,wind_direction,wind_speed,timerecorded,source
691,5.702,0.3,12.42,6.0,5.282,2020-06-12 00:00:00,GAN
690,7.077,0.0,9.99,7.5,4.806,2020-06-12 01:00:00,GAN
689,7.881,0.0,13.88,8.3,3.607,2020-06-12 02:00:00,GAN
688,8.036,0.0,11.64,216.8,2.554,2020-06-12 03:00:00,GAN
687,7.920,0.0,10.77,10.5,4.524,2020-06-12 04:00:00,GAN
...,...,...,...,...,...,...,...
4,2.690,0.0,16.24,264.3,5.898,2020-07-11 10:00:00,SENSOR
3,3.978,0.0,16.78,263.3,5.929,2020-07-11 11:00:00,SENSOR
2,5.659,0.0,16.62,266.8,5.007,2020-07-11 12:00:00,SENSOR
1,7.019,0.0,16.45,268.0,5.341,2020-07-11 13:00:00,SENSOR


In [20]:
#reset file index to current dataframe 

final_file_1_hr.reset_index(drop=True, inplace=True)

In [21]:
#save as a csv file locally

final_file_1_hr.to_csv("final_data_1_hr.csv")

In [22]:
import datetime
import tarfile

import boto3 # AWS SDK for python. Provides low-level access to AWS services
from sagemaker import get_execution_role
import sagemaker

m_boto3 = boto3.client('sagemaker') 

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = 'flood-prediction-master-dataset'  #  Bucket name where the file is to be saved

print('Using bucket ' + bucket)

# save data to S3. key_prefix is the directory path. path is the local file name which will be saved in S3. It has to be 
# the same file name as saved locally and bucket is the bucket name 

upload = sess.upload_data(path='final_data_1_hr.csv', bucket=bucket, key_prefix='final-dataset-1-hr')

Using bucket flood-prediction-master-dataset


In [28]:
print("Mean of River GAN: "+str(round(gan_file['river'].mean(),4))+" Mean of River Sensor: "+str(round(sensor_file['river'].mean(),4)))

print("Mean of Rainfall GAN: "+str(round(gan_file['rain'].mean(),4))+" Mean of Rainfall Sensor: "+str(round(sensor_file['rain'].mean(),4)))

print("Mean of Temperature GAN: "+str(round(gan_file['temperature'].mean(),4))+" Mean of Temperature Sensor: "+str(round(sensor_file['temperature'].mean(),4)))

print("Mean of wind direction GAN: "+str(round(gan_file['wind_direction'].mean(),4))+" Mean of wind direction Sensor: "+str(round(sensor_file['wind_direction'].mean(),4)))

print("Mean of wind speed GAN: "+str(round(gan_file['wind_speed'].mean(),4))+" Mean of wind speed Sensor: "+str(round(sensor_file['wind_speed'].mean(),4)))

Mean of River GAN: 5.807 Mean of River Sensor: 5.7617
Mean of Rainfall GAN: 0.1603 Mean of Rainfall Sensor: 0.1474
Mean of Temperature GAN: 15.6879 Mean of Temperature Sensor: 15.6435
Mean of wind direction GAN: 182.7709 Mean of wind direction Sensor: 182.8452
Mean of wind speed GAN: 5.4265 Mean of wind speed Sensor: 5.4481
